# Biomethane-to-Ammonia - Import LCI

This notebook imports the life cycle inventories (LCIs) and creates regionalized inventories for the European Union

In [1]:
%run _imports.ipynb

### Import inventories

In [2]:
# Import ecoinvent database into wurst format (i.e., list of dictionary, each dict being a dataset)
try:
  len(ei_db)
except NameError:
  ei_db = wurst.extract_brightway2_databases(ECOINVENT_DB)

# Import the biosphere database as a list of dictionary
biosphere_db = [ef.as_dict() for ef in bw.Database('biosphere3')]

Getting activity data


100%|██████████| 21238/21238 [00:00<00:00, 206124.19it/s]


Adding exchange data to activities


100%|██████████| 674593/674593 [01:26<00:00, 7839.42it/s] 


Filling out exchange data


100%|██████████| 21238/21238 [00:04<00:00, 4751.63it/s] 


**! IMPORTANT !**

Some of the LCI datasets in the inventories file are partially based on data from the ecoinvent LCI database. To comply with licensing requirements, 
the file shared in this repository does not include these data points. Instead, a "PLACEHOLDER" string has been included. This will cause an error when trying to
import the LCIs.

If you hold a valid ecoinvent license, please contact me directly to receive the full input files containing all ecoinvent data points.

In [3]:
LCI_PATH = Path(DATA_DIR / "inventories.xlsx")

# Import inventories that are already in BW2 format (i.e., biogas production, biomethane market, and green ammonia)
lci_import = bw.ExcelImporter(LCI_PATH)
lci_import.apply_strategies()
lci_import.match_database(ECOINVENT_DB, fields=('name', 'reference product', 'unit', 'location'))
lci_import.match_database("biosphere3", fields=('name', 'unit', 'categories'))

Extracted 5 worksheets in 0.23 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 5.72 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields


In [4]:
lci_db = []
lci_db = lci_import.data

# Create additional inventories from dataframes
# Excel sheets with inventories in dataframe format
LCI_SHEETS = ['LCI_Upgrading',
              'LCI_Heat',
              'LCI_Ammonia_NG&Biomethane']
for sheet in LCI_SHEETS:
    inventories_df = pd.read_excel(LCI_PATH, sheet_name=sheet, skiprows=1).fillna(0)
    lci_converted = inventory_imports.create_dataset_from_df(inventories_df)
    lci_db += lci_converted

In [5]:
# The ExcelImporter requires that a 'reference product' is defined for every technosphere exchange.
# However, the ecoinvent database imported in BW2 as well as wurst use 'product' for technosphere exchanges.
# Replace 'reference product' by 'product' in technosphere exchanges.
inventory_imports.correct_product_in_exchanges(lci_db)

In [6]:
# Link technosphere and biosphere exchanges by code
inventory_imports.link_exchanges_by_code(lci_db, ei_db, biosphere_db)

### Create regionalized inventories for EU

In [7]:
# Import biomethane data by country
biomethane_potential = pd.read_excel(Path(DATA_DIR / "sustainable_biomethane_potential_Europe.xlsx"),
                                     sheet_name='Biomethane Potential',
                                     usecols='A:AE',
                                     index_col=[0])

# Country-specific market for biogas
biogas_market_by_country = biomethane_potential.loc['Sustainable biomethane share by feedstock in 2030'].reset_index(
                                                    ).drop(['Parameter', 'Unit'], axis=1).set_index(['Feedstock']).T
biogas_market_by_country.head()

Feedstock,Animal manure,Agricultural residues,Sequential crops,Industrial wastewater,Biowaste,Sewage sludge
Total,0.344,0.258,0.226,0.097,0.054,0.022
DE,0.312,0.278,0.181,0.164,0.032,0.033
FR,0.339,0.271,0.251,0.111,0.009,0.019
IT,0.192,0.095,0.585,0.046,0.062,0.020
ES,0.353,0.236,0.235,0.099,0.042,0.035


In [8]:
LIST_COUNTRIES = [c for c in biogas_market_by_country.index if c != 'Total']
LIST_COUNTRIES[:4]

['DE', 'FR', 'IT', 'ES']

In [9]:
ds_to_regionalize = [
                     ('anaerobic digestion of animal manure, with biogenic carbon uptake', 'biogas', 'RER'),
                     ('anaerobic digestion of agricultural residues, with biogenic carbon uptake', 'biogas', 'RER'),
                     ('anaerobic digestion of sequential crop, with biogenic carbon uptake', 'biogas', 'RER'),
                     ('market for biogas, sustainable feedstocks', 'biogas', 'RER'),

                     ('biogas upgrading to biomethane, water scrubbing', 'biomethane, 24 bar', 'RER'),
                     ('biogas upgrading to biomethane, water scrubbing w/ CCS', 'biomethane, 24 bar', 'RER'),
                     ('biogas upgrading to biomethane, chemical scrubbing', 'biomethane, 24 bar', 'RER'),
                     ('biogas upgrading to biomethane, chemical scrubbing w/ CCS', 'biomethane, 24 bar', 'RER'),
                     ('biogas upgrading to biomethane, membrane', 'biomethane, 24 bar', 'RER'),
                     ('biogas upgrading to biomethane, membrane w/ CCS', 'biomethane, 24 bar', 'RER'),
                     ('biogas upgrading to biomethane, pressure swing adsorption', 'biomethane, 24 bar', 'RER'),
                     ('biogas upgrading to biomethane, pressure swing adsorption w/ CCS', 'biomethane, 24 bar', 'RER'),
                     ('market for biomethane, 24 bar', 'biomethane, 24 bar', 'RER'),
                     ('market for biomethane, 24 bar w/ CCS', 'biomethane, 24 bar', 'RER'),

                     ('carbon dioxide capture, chemical absorption, with transport and storage, 200 km pipeline sotage 1000m',
                      'carbon dioxide capture, chemical absorption, with transport and storage, 200 km pipeline sotage 1000m', 'RER'),
                     
                     ('heat production, at industrial furnace >100kW, natural gas, heating', 'heat, district or industrial', 'RER'),
                     ('heat production, at industrial furnace >100kW, natural gas, heating w/ CCS', 'heat, district or industrial', 'RER'),
                     ('heat production, at industrial furnace >100kW, biomethane', 'heat, district or industrial', 'RER'),  
                     ('heat production, at industrial furnace >100kW, biomethane, heating w/ CCS', 'heat, district or industrial', 'RER'),
                     ('heat production, at industrial furnace >100kW, biomethane w/ CCS', 'heat, district or industrial', 'RER'),
                     ('heat production, at industrial furnace >100kW, biomethane w/ CCS, heating w/ CCS', 'heat, district or industrial', 'RER'),

                     ('ammonia production, liquid, fossil ammonia from natural gas', 'ammonia, anhydrous, liquid', 'RER'),
                     ('ammonia production, liquid, fossil ammonia from natural gas, syngas w/ CCS', 'ammonia, anhydrous, liquid', 'RER'),
                     ('ammonia production, liquid, fossil ammonia from natural gas, syngas w/ CCS, heating w/ CCS', 'ammonia, anhydrous, liquid', 'RER'),

                     ('ammonia production, liquid, bio-ammonia from biomethane', 'ammonia, anhydrous, liquid', 'RER'),
                     ('ammonia production, liquid, bio-ammonia from biomethane, syngas w/ CCS', 'ammonia, anhydrous, liquid', 'RER'),
                     ('ammonia production, liquid, bio-ammonia from biomethane, syngas w/ CCS, heating w/ CCS', 'ammonia, anhydrous, liquid', 'RER'),
                     ('ammonia production, liquid, bio-ammonia from biomethane w/ CCS', 'ammonia, anhydrous, liquid', 'RER'),
                     ('ammonia production, liquid, bio-ammonia from biomethane w/ CCS, syngas w/ CCS', 'ammonia, anhydrous, liquid', 'RER'),
                     ('ammonia production, liquid, bio-ammonia from biomethane w/ CCS, syngas w/ CCS, heating w/ CCS', 'ammonia, anhydrous, liquid', 'RER')
                    ]

In [10]:
lci_db_regional = inventory_imports.regionalize_inventories(ds_to_regionalize,
                                                            LIST_COUNTRIES,
                                                            ei_db + lci_db,
                                                            lci_db[0]['database'])

In [11]:
# Update country-specific biogas markets
map_feedstock = {
        'treatment of biowaste by anaerobic digestion, cut-off with biogenic carbon uptake':              'Biowaste',
        'treatment of industrial wastewater by anaerobic digestion, cut-off with biogenic carbon uptake': 'Industrial wastewater',
        'treatment of sewage sludge by anaerobic digestion, cut-off with biogenic carbon uptake':         'Sewage sludge',
        'anaerobic digestion of animal manure, with biogenic carbon uptake':                              'Animal manure',
        'anaerobic digestion of agricultural residues, with biogenic carbon uptake':                      'Agricultural residues',
        'anaerobic digestion of sequential crop, with biogenic carbon uptake':                            'Sequential crops'
    }

technosphere = lambda x: x["type"] == "technosphere"
for ds in filter(lambda ds: ds["name"] == 'market for biogas, sustainable feedstocks', lci_db_regional):
    for exc in filter(technosphere, ds["exchanges"]):
        biogas_share = biogas_market_by_country.loc[ds['location']][map_feedstock[exc['name']]]
        exc.update({'amount': biogas_share})

### Write database

In [14]:
LCI_DB_NAME = 'biomethane-to-ammonia'
if LCI_DB_NAME in bw.databases:
    del bw.databases[LCI_DB_NAME]
wurst.write_brightway2_database(lci_db + lci_db_regional, LCI_DB_NAME)

855 datasets
14133 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Title: Writing activities to SQLite3 database:
  Started: 01/01/2024 21:43:36
  Finished: 01/01/2024 21:43:38
  Total time elapsed: 00:00:01
  CPU %: 99.10
  Memory %: 34.97
Created database: biomethane-to-ammonia
